In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('/kaggle/input/obs-data/New_OBS_DATA_FOR_ML.csv')
df.info()

In [ ]:
features = list(df.columns)
features.remove('TC_or_NTC')

print('特徵數: ', len(features))
features

In [ ]:
print('數值型特徵數: ', df[features].describe().shape[1]-1)
df.describe()

In [ ]:
# 相關係數矩陣
cor_matrix = df.corr(numeric_only=True)
cor_matrix

In [ ]:
import seaborn as sns

# HeatMap of Correlation Matrix
plt.figure(figsize=(40,40))
mask = np.triu(np.ones_like(cor_matrix, dtype=bool))
sns.heatmap(cor_matrix, annot=True, mask=mask, vmin=-1, vmax=1)
plt.title('Correlation Coefficient of Predictors')
plt.show()

In [ ]:
# 篩選相關係數大於 0.8 的特徵
thresh = 0.8
mask = abs(cor_matrix) > thresh
cor_matrix.where(mask).stack()

In [ ]:
def compute_vif(considered_features):

    X = df.loc[:,considered_features]
    X = X.dropna()
    X['intercept'] = 1

    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif.loc[vif['Variable']!='intercept']

    return vif

In [ ]:
# 考慮要刪除的特徵
considered_features = ['dtd8', 'dtd9', 'dtd10', 'dtd11', 'dtd12', 'dtd13', 
           'rh8', 'rh9', 'rh10', 'rh11', 'rh12', 'rh13', 
           'M8', 'M9', 'M10', 'M11', 'M12', 'M13']

# 計算 VIF
compute_vif(considered_features).sort_values('VIF', ascending=False)

In [ ]:
# 刪除 'Horsepower' 與 Power_perf_factor' 後計算 VIF
considered_features = ['dtd8', 'dtd9', 'dtd10', 'dtd11', 'dtd12', 'dtd13', 
           'rh8', 'rh9', 'rh10', 'rh11', 'rh12', 'rh13', 
           'M8', 'M9', 'M10', 'M11', 'M12', 'M13']

for f in ['dtd8', 'dtd9', 'dtd10', 'dtd11', 'dtd12', 'dtd13', 
           'rh8', 'rh9', 'rh10', 'rh11', 'rh12', 'rh13', 'M9', 'M10', 'M11']:
    considered_features.remove(f)

compute_vif(considered_features).sort_values('VIF', ascending=False)

In [ ]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import KFold
kfold = KFold(n_splits=4, shuffle=True)

In [ ]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)
def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

In [ ]:
class OBSDataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else: 
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

In [ ]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 31),
            nn.BatchNorm1d(31),
            nn.LeakyReLU(),
            nn.Linear(31, 16),
            nn.BatchNorm1d(16),
            nn.LeakyReLU(),
            nn.Linear(16, 8),
            nn.BatchNorm1d(8),
            nn.LeakyReLU(),            
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        #x = x.squeeze(1) # (B, 1) -> (B)
        return x

In [ ]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,27,31,32,33,34,35,36,37,38,39,40,41,42,43,44] # TODO: Select suitable feature columns.
        len_feat_idx = len(feat_idx)
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid, len_feat_idx

In [ ]:
def trainer(train_loader, valid_loader, model, config, device):
    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.
    #criterion = nn.L1Loss() 
    
    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    warmup_steps = 50
    #optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9) 
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])
    #scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, config['n_epochs'])                              
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0
    loss_train=[]; loss_val=[]
    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)
        for x, y in train_loader:
            for x, y in train_pbar:
                optimizer.zero_grad()               # Set gradient to zero.
                y = torch.reshape(y,(len(y),1))
                x, y = x.to(device), y.to(device)   # Move your data to device.
                pred = model(x)             
                loss = criterion(pred, y)
                loss.backward()                     # Compute gradient(backpropagation).
                optimizer.step()                    # Update parameters.
                #scheduler.step()
                step += 1
                loss_record.append(loss.detach().item())

                # Display current epoch number and loss on tqdm progress bar.
                train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
                train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            y = torch.reshape(y,(len(y),1))
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        loss_train.append(mean_train_loss);loss_val.append(mean_valid_loss);
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss :#and mean_valid_loss<0.123:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return loss_train, loss_val

In [ ]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
        print(f'Reset trainable parameters of layer = {layer}')
        layer.reset_parameters()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'valid_ratio': 0.1,   # validation_size = train_size * valid_ratio
    'n_epochs': 100,     # Number of epochs.            
    'batch_size': 128, 
    'learning_rate': 1e-5,              
    'early_stop': 50,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
    # Process features within a batch.
    """Collate a batch of data."""
    x, y = zip(*batch)
    #print(x)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    x = pad_sequence(x, batch_first=True)#, padding_value=-20)    # pad log 10^(-20) which is very small value.
    #print(x.shape)
    # mel: (batch size, length, 40)
    return x, torch.FloatTensor(y)#.long()

In [ ]:
# Set seed for reproducibility
same_seed(config['seed'])

train_data, test_data = pd.read_csv('../input/obs-data/New_OBS_DATA_FOR_ML.csv').values, pd.read_csv('../input/obs-data/New_test_OBS_DATA_FOR_ML.csv').values

# K-fold Cross Validation model evaluation
#for fold, (train_ids, test_ids) in enumerate(kfold.split(train_data)):
#    print(f'FOLD {fold}')
#    print('--------------------------------') 
#train_data_f = torch.tensor([train_data[i] for i in train_ids],dtype=torch.float32)
#valid_data_f = torch.tensor([train_data[j] for j in test_ids],dtype=torch.float32)
#test_data = torch.tensor(test_data,dtype=torch.float32)

train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid, len_feat_idx = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = OBSDataset(x_train, y_train), \
                        OBSDataset(x_valid, y_valid), \
                        OBSDataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)#,drop_last=True,collate_fn=collate_batch)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)#,drop_last=True,collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)#,drop_last=True,collate_fn=collate_batch)

model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
#model.apply(reset_weights)
a, b = trainer(train_loader, valid_loader, model, config, device)

In [ ]:
#pip install shap==0.39.0
#!conda list -n shap
#!pip install shapely==1.7.1

In [ ]:
import torch
import shap
import numpy as np

saved_model_path = config['save_path']
checkpoint = torch.load(saved_model_path)

# Step 4: Load the parameters into the model
model.load_state_dict(checkpoint)
print(model)

batch = next(iter(test_loader))
print(batch)
x = batch#[0:100]
x = x.to(device)
print(x.shape, type(model), type(x))
#x = torch.reshape(x,(1,178,31))
#print(x.shape)
#x = torch.permute(x, (2, 1, 0))
#print(x.shape)
e = shap.DeepExplainer(model, x)  

# Get the shap values from my test data (this explainer likes tensors)
shap_values = e.shap_values(x) 

# Plots
#shap.force_plot(explainer.expected_value, shap_values, feature_names)
#shap.dependence_plot("b1_price_avg", shap_values, data, feature_names)
feature_names = [ 'lat', 'lon', 'alt', 
           'p9', 'p10', 'p11', 'p12',
           't9', 't10', 't11', 't12', 
           'dtd9', 'dtd10', 'dtd11', 'dtd12', 
           'rh9', 'rh10', 'rh11', 'rh12', 
           'M9', 'M10', 'M11', 'M12', 
           'wd9', 'wd10', 'wd11', 'wd12',  
           'ws9', 'ws10', 'ws11', 'ws12'
         ]
shap.summary_plot(shap_values, x, plot_type='bar',feature_names=feature_names)

In [ ]:
import matplotlib.pyplot as plt
a = np.asarray(a)
print(a.shape)
len_epoch = np.linspace(0,len(a),len(a))
plt.plot(len_epoch,a)
plt.plot(len_epoch,b)
plt.legend(['train_loss','valid_loss'])
plt.show()

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'TC/NTC (1/0)'])
        for i, p in enumerate(preds):
            if (p>=0.5):
              p = 1
            else:
              p=0
            writer.writerow([i, p])
#test_data = torch.tensor(test_data,dtype=torch.float32)
#x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data_f, valid_data_f, test_data, config['select_all'])
#test_dataset = OBSDataset(x_test)
#test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')      